In [1]:
!pip install trimesh

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 688.5/688.5 kB 8.8 MB/s eta 0:00:00


In [8]:
!pip3 install torch
!pip3 install torchvision

In [9]:
import os
import glob
import trimesh
import numpy as np
import tensorflow as tf
import torch


import plotly.graph_objects as go
import plotly.express as px

from matplotlib import pyplot as plt

tf.random.set_seed(1234)

NameError: ignored

In [10]:
def pcshow(xs,ys,zs):
    data=[go.Scatter3d(x=xs, y=ys, z=zs,
                                   mode='markers')]
    fig = visualize_rotate(data)
    fig.update_traces(marker=dict(size=2,
                      line=dict(width=2,
                      color='DarkSlateGrey')),
                      selector=dict(mode='markers'))
    fig.show()


def Normalize(pointcloud):
    norm_pointcloud = pointcloud - np.mean(pointcloud, axis=0)
    norm_pointcloud /= np.max(np.linalg.norm(norm_pointcloud, axis=1))

    return  norm_pointcloud

def visualize_rotate(data):
    x_eye, y_eye, z_eye = 1., 1., 1.
    frames=[]

    def rotate_z(x, y, z, theta):
        w = x+1j*y
        return np.real(np.exp(1j*theta)*w), np.imag(np.exp(1j*theta)*w), z

    for t in np.arange(0, 10.26, 0.1):
        xe, ye, ze = rotate_z(x_eye, y_eye, z_eye, -t)
        frames.append(dict(layout=dict(scene=dict(camera=dict(eye=dict(x=xe, y=ye, z=ze))))))
    fig = go.Figure(data=data,
                    layout=go.Layout(
                        updatemenus=[dict(type='buttons',
                                    showactive=False,
                                    y=1,
                                    x=0.8,
                                    xanchor='left',
                                    yanchor='bottom',
                                    pad=dict(t=45, r=10),
                                    buttons=[dict(label='Play',
                                                    method='animate',
                                                    args=[None, dict(frame=dict(duration=50, redraw=True),
                                                                    transition=dict(duration=0),
                                                                    fromcurrent=True,
                                                                    mode='immediate'
                                                                    )]
                                                    )
                                            ]
                                    )
                                ]
                    ),
                    frames=frames
            )

    return fig


In [11]:
mesh = trimesh.load('/content/drive/MyDrive/이빨/data/1_normal/normal_1.stl')
mesh.show()

In [12]:
vertices = np.array(mesh.vertices)
faces = np.array(mesh.faces)

In [13]:
pcshow(vertices[:, 0], vertices[:, 1], vertices[:, 2])

In [14]:
import random

class PointSampler(object):
    def __init__(self, output_size):
        assert isinstance(output_size, int)
        self.output_size = output_size

    def triangle_area(self, pt1, pt2, pt3):
        side_a = np.linalg.norm(pt1 - pt2)
        side_b = np.linalg.norm(pt2 - pt3)
        side_c = np.linalg.norm(pt3 - pt1)
        s = 0.5 * ( side_a + side_b + side_c)
        return max(s * (s - side_a) * (s - side_b) * (s - side_c), 0)**0.5

    def sample_point(self, pt1, pt2, pt3):
        # barycentric coordinates on a triangle
        # https://mathworld.wolfram.com/BarycentricCoordinates.html
        s, t = sorted([random.random(), random.random()])
        f = lambda i: s * pt1[i] + (t-s)*pt2[i] + (1-t)*pt3[i]
        return (f(0), f(1), f(2))


    def __call__(self, mesh):
        verts, faces = mesh
        verts = np.array(verts)
        areas = np.zeros((len(faces)))

        for i in range(len(areas)):
            areas[i] = (self.triangle_area(verts[faces[i][0]],
                                           verts[faces[i][1]],
                                           verts[faces[i][2]]))

        sampled_faces = (random.choices(faces,
                                      weights=areas,
                                      cum_weights=None,
                                      k=self.output_size))

        sampled_points = np.zeros((self.output_size, 3))

        for i in range(len(sampled_faces)):
            sampled_points[i] = (self.sample_point(verts[sampled_faces[i][0]],
                                                   verts[sampled_faces[i][1]],
                                                   verts[sampled_faces[i][2]]))

        return sampled_points

In [15]:
pointcloud = PointSampler(2048)((vertices, faces))

In [16]:
fig = plt.figure(figsize=(5, 5))
ax = fig.gca(projection="3d")
ax.scatter(pointcloud[:, 0], pointcloud[:, 1], pointcloud[:, 2])
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('z')
plt.show()

TypeError: ignored

<Figure size 500x500 with 0 Axes>

In [17]:
pcshow(pointcloud[:, 0],pointcloud[:, 1],pointcloud[:, 2])

In [18]:
os.getcwd()

'/content'

In [19]:
DATA_DIR ='/content/drive/MyDrive/이빨/data/'
pointcloud = []
label = []
class_map = {}
folders = os.listdir(DATA_DIR)

In [20]:
from natsort import natsorted

In [21]:
folders = natsorted(folders)

In [22]:
num_points = 2048

In [23]:
for i, folder in enumerate(folders):
    print("processing class: {}".format(os.path.basename(folder)))
    class_map[i] = folder
    files_name = os.listdir(DATA_DIR + folder)
    for f in files_name:
        mesh = trimesh.load(DATA_DIR + folder + '/' +f)
        mesh_vertices = np.array(mesh.vertices)
        mesh_faces = np.array(mesh.faces)
        pointcloud.append(Normalize(PointSampler(num_points)((mesh_vertices, mesh_faces))))
        label.append(i)

processing class: 1_normal
processing class: 2_crown
processing class: 3_implant


In [24]:
import os
len(os.listdir('/content/drive/MyDrive/이빨/data/1_normal'))

254

In [25]:
import os
len(os.listdir('/content/drive/MyDrive/이빨/data/2_crown'))

131

In [26]:
import os
len(os.listdir('/content/drive/MyDrive/이빨/data/3_implant'))

45

In [27]:
class_map

{0: '1_normal', 1: '2_crown', 2: '3_implant'}

In [28]:
import pickle

# Save Pickle point 1024
with open('pointcloud_1024.txt', 'wb') as f:
     pickle.dump(pointcloud, f)

# Save Pickle point 1024
with open('label_1024.txt', 'wb') as f:
     pickle.dump(label, f)

In [29]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(pointcloud, label, test_size=0.2, stratify = label, random_state=42)

In [30]:
import numpy as np

X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

In [31]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow as tf

In [32]:
import torch
import torch.nn as nn
import torch.optim as optim

class PointCloudTransformer(nn.Module):
    def __init__(self, input_size, num_classes, num_heads=8, hidden_size=256):
        super(PointCloudTransformer, self).__init__()

        self.embedding = nn.Linear(input_size, hidden_size)
        self.transformer_layer = nn.TransformerEncoderLayer(
            d_model=hidden_size,
            nhead=num_heads
        )
        self.transformer = nn.TransformerEncoder(
            self.transformer_layer,
            num_layers=3
        )
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        x = self.embedding(x)
        x = x.permute(1, 0, 2)  # Change the shape for transformer input
        x = self.transformer(x)
        x = x.mean(dim=0)  # Global average pooling
        x = self.fc(x)
        return x

In [33]:
# 모델 초기화
input_size = 3
num_classes = 3
model = PointCloudTransformer(input_size=input_size, num_classes=num_classes)

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:282: UserWarning:

enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)



In [34]:
# 손실 함수와 최적화 알고리즘 설정
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [35]:
# PyTorch Tensor로 변환
X_train_tensor = torch.Tensor(X_train)
y_train_tensor = torch.Tensor(y_train).type(torch.long)
X_val_tensor = torch.Tensor(X_test)
y_val_tensor = torch.Tensor(y_test).type(torch.long)

In [36]:
num_epochs = 10

In [38]:
inputs = keras.Input(shape=(num_points, 3))

In [41]:
!pip install torch torchvision torchaudio
import torch
print(torch.cuda.is_available())

True


In [42]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

PointCloudTransformer(
  (embedding): Linear(in_features=3, out_features=256, bias=True)
  (transformer_layer): TransformerEncoderLayer(
    (self_attn): MultiheadAttention(
      (out_proj): NonDynamicallyQuantizableLinear(in_features=256, out_features=256, bias=True)
    )
    (linear1): Linear(in_features=256, out_features=2048, bias=True)
    (dropout): Dropout(p=0.1, inplace=False)
    (linear2): Linear(in_features=2048, out_features=256, bias=True)
    (norm1): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
    (norm2): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
    (dropout1): Dropout(p=0.1, inplace=False)
    (dropout2): Dropout(p=0.1, inplace=False)
  )
  (transformer): TransformerEncoder(
    (layers): ModuleList(
      (0-2): 3 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=256, out_features=256, bias=True)
        )
        (linear1): Linear(in_features=256, out_features

In [46]:
X_train_tensor = X_train_tensor.to(device)
y_train_tensor = y_train_tensor.to(device)
X_val_tensor = X_val_tensor.to(device)
y_val_tensor = y_val_tensor.to(device)

In [47]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import classification_report, f1_score
import numpy as np

# 모델 정의 및 초기화
input_size = 3
num_classes = 3

model = PointCloudTransformer(input_size=input_size, num_classes=num_classes)
model.to(device)


# 손실 함수와 최적화 알고리즘 설정
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 훈련 데이터를 DataLoader로 변환
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)



# 검증 데이터를 DataLoader로 변환
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
val_loader = DataLoader(val_dataset, batch_size=32)

# 에폭 수 설정
num_epochs = 10

# 초기 best 모델 설정
best_val_loss = float('inf')
best_model_weights = None

# 훈련 루프
for epoch in range(num_epochs):
    # 훈련
    model.train()
    for inputs, labels in train_loader:
        inputs = inputs.view(-1, 2048, 3)
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # 검증
    model.eval()
    with torch.no_grad():
        val_loss = 0.0
        val_predictions = []
        val_true_labels = []

        for val_inputs, val_labels in val_loader:
            val_outputs = model(val_inputs)
            val_loss += criterion(val_outputs, val_labels).item()

            _, val_predicted = torch.max(val_outputs, 1)
            val_predictions.extend(val_predicted.cpu().numpy())
            val_true_labels.extend(val_labels.cpu().numpy())

        val_loss /= len(val_loader)

        # Validation Classification Report 및 F1-Score 계산
        val_classification_report = classification_report(val_true_labels, val_predictions)
        val_f1 = f1_score(val_true_labels, val_predictions, average='weighted')

    # 에폭마다 출력
    print(f'Epoch [{epoch + 1}/{num_epochs}], Train Loss: {loss.item():.4f}, Val Loss: {val_loss:.4f}')
    print("Validation Classification Report:\n", val_classification_report)
    print("Validation Weighted F1-Score:", val_f1)

    # 최고의 모델 갱신 및 저장
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_model_weights = model.state_dict()
        torch.save(best_model_weights, 'best_model_weights.pth')

print("Training complete.")

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:282: UserWarning:

enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_divis

Epoch [1/10], Train Loss: 0.8911, Val Loss: 0.9204
Validation Classification Report:
               precision    recall  f1-score   support

           0       0.59      1.00      0.74        51
           1       0.00      0.00      0.00        26
           2       0.00      0.00      0.00         9

    accuracy                           0.59        86
   macro avg       0.20      0.33      0.25        86
weighted avg       0.35      0.59      0.44        86

Validation Weighted F1-Score: 0.44152096418265147


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



Epoch [2/10], Train Loss: 0.7100, Val Loss: 0.9153
Validation Classification Report:
               precision    recall  f1-score   support

           0       0.59      1.00      0.74        51
           1       0.00      0.00      0.00        26
           2       0.00      0.00      0.00         9

    accuracy                           0.59        86
   macro avg       0.20      0.33      0.25        86
weighted avg       0.35      0.59      0.44        86

Validation Weighted F1-Score: 0.44152096418265147


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



...

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



Epoch [8/10], Train Loss: 1.0359, Val Loss: 0.9232
Validation Classification Report:
               precision    recall  f1-score   support

           0       0.59      1.00      0.74        51
           1       0.00      0.00      0.00        26
           2       0.00      0.00      0.00         9

    accuracy                           0.59        86
   macro avg       0.20      0.33      0.25        86
weighted avg       0.35      0.59      0.44        86

Validation Weighted F1-Score: 0.44152096418265147


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



Epoch [9/10], Train Loss: 0.9136, Val Loss: 0.9136
Validation Classification Report:
               precision    recall  f1-score   support

           0       0.59      1.00      0.74        51
           1       0.00      0.00      0.00        26
           2       0.00      0.00      0.00         9

    accuracy                           0.59        86
   macro avg       0.20      0.33      0.25        86
weighted avg       0.35      0.59      0.44        86

Validation Weighted F1-Score: 0.44152096418265147
Epoch [10/10], Train Loss: 0.9832, Val Loss: 0.9117
Validation Classification Report:
               precision    recall  f1-score   support

           0       0.59      1.00      0.74        51
           1       0.00      0.00      0.00        26
           2       0.00      0.00      0.00         9

    accuracy                           0.59        86
   macro avg       0.20      0.33      0.25        86
weighted avg       0.35      0.59      0.44        86

Validation Weig

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



In [30]:
from torch.utils.data import DataLoader, TensorDataset

# X_train_tensor, y_train_tensor를 TensorDataset으로 묶고 DataLoader로 미니배치 생성
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

# 훈련 루프에서 DataLoader 사용
for epoch in range(num_epochs):
    for inputs, labels in train_loader:
        outputs = model(inputs)
        loss = criterion(outputs, labels)

KeyboardInterrupt: ignored

In [ ]:
from sklearn.metrics import classification_report, f1_score
import numpy as np

# 모델 예측
with torch.no_grad():
    X_test_tensor = torch.Tensor(X_test)
    outputs = model(X_test_tensor)
    _, predicted = torch.max(outputs, 1)

# 실제 라벨과 예측 결과를 numpy 배열로 변환
y_true = np.array(y_test)
y_pred = predicted.cpu().numpy()  # GPU에서 예측한 결과를 CPU로 이동 후 numpy 배열로 변환

# Classification Report 출력
print("Classification Report:\n", classification_report(y_true, y_pred))

# F1-Score 출력
f1 = f1_score(y_true, y_pred, average='weighted')
print("Weighted F1-Score:", f1)

In [ ]:
torch.save(model.state_dict(), 'PCT_weights.pth')